### Read data

Dziękuję za dokumentację wstępną. Cieszę się żę uwzględnili Państwo moje uwagi odnośnie dodatkowych algorytmów optymalizacji.
Z drobnych uwag to polecałbym skorzystać z gotowej biblioteki do algorytmów genetycznych np. PyGAD
W dokumentacji brakuje mi informacji na temat podziału zbioru danych na treningowy (na którym otpymalizować powinni Państwo knn-a, walidacyjnym na którym obliczane będą metryki i testowy - który wykorzystany będzie do wyznaczenia końcowej jakości rozwiązania)

In [14]:
from sklearn.datasets import load_svmlight_file
import pandas as pd
from time import time

def get_data():
    # Assuming 'farm-ads-vect' is in SVMlight format
    X, y = load_svmlight_file('data/farm-ads-vect')

    # Convert sparse matrix to DataFrame
    df_vectors = pd.DataFrame.sparse.from_spmatrix(X)

    return df_vectors, y

In [2]:
df, labels = get_data()

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,54867,54868,54869,54870,54871,54872,54873,54874,54875,54876
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df.shape

(4143, 54877)

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score

def evaluate_model(model, X_test, y_test):
    """
    Evaluate a logistic regression model on the test data.

    Parameters:
    - model: trained logistic regression model
    - X_test: DataFrame, feature vectors for testing
    - y_test: Series, true labels for testing

    Returns:
    - accuracy: float, accuracy of the model on the test set
    - report: str, classification report (includes precision, recall, f1-score, and support)
    """
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return accuracy, f1


def train_knn(X_train, y_train, n_neighbors=5):
    """
    Train a k-nearest neighbors (KNN) model on the training data.

    Parameters:
    - X_train: DataFrame, feature vectors for training
    - y_train: Series, labels for training
    - n_neighbors: int, number of neighbors to consider (default is 5)

    Returns:
    - model: trained KNN model
    """
    model = KNeighborsClassifier(n_neighbors=n_neighbors)
    model.fit(X_train, y_train)
    return model

In [40]:
from typing import List


def evaluate_attribute_set(attribute_names: List[str], df, labels):
    df_temp = df[attribute_names]

    X_train, X_val, y_train, y_val = train_test_split(df_temp, labels, test_size=0.2, random_state=42)

    # Train the logistic regression model
    model = train_knn(X_train, y_train)

    # Evaluate the model
    accuracy, f1 = evaluate_model(model, X_val, y_val)

    return accuracy, f1

In [46]:
def sequential_forward_selection(df, labels, max_features=None):
    """
    Perform Sequential Forward Selection (SFS) for feature selection.

    Parameters:
    - X_train: DataFrame, feature vectors for training
    - y_train: Series, labels for training
    - X_test: DataFrame, feature vectors for testing
    - y_test: Series, true labels for testing
    - max_features: int, maximum number of features to select (default is None)

    Returns:
    - best_features: list, selected features
    """
    num_features = df.shape[1]
    all_features = list(df.columns)
    selected_features = []
    best_accuracy = 0.0
    best_f1 = 0.0

    df_performance = pd.DataFrame(columns=["selected_features", "num_selected_features", "time", "accuracy_validate"])
    start_time = time()
    while len(selected_features) < num_features and (max_features is None or len(selected_features) < max_features):
        remaining_features = [feature for feature in all_features if feature not in selected_features]
        current_best_feature = None

        for feature in remaining_features:
            trial_features = selected_features + [feature]
            accuracy, f1 = evaluate_attribute_set(trial_features, df, labels)

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_f1 = f1
                current_best_feature = feature

        if current_best_feature is not None:
            selected_features.append(current_best_feature)
            print(f"Selected Features: {selected_features}")
            print(f"Accuracy with Selected Features: {best_accuracy:.2f}")
            df_performance = df_performance.append({
                "selected_features": selected_features.copy(),
                "num_selected_features": len(selected_features),
                "time": time() - start_time,
                "accuracy_validate": best_accuracy,
                "f1_validate": best_f1
            }, ignore_index=True)

    return selected_features, best_accuracy, best_f1, df_performance

In [47]:
N_OF_FEATURES_TO_TEST = 100

In [48]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:N_OF_FEATURES_TO_TEST], labels, test_size=0.2, random_state=42)

In [ ]:
selected_features, best_accuracy, best_f1, df_performance = sequential_forward_selection(X_train, y_train, max_features=5)
selected_features, best_accuracy

Selected Features: [15]
Accuracy with Selected Features: 0.64


C:\Users\PB\AppData\Local\Temp\ipykernel_35860\1462941052.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_performance = df_performance.append({


In [38]:
df_performance

,selected_features,num_selected_features,time,accuracy
0,[38],1,12.350553,0.720145
1,"[38, 15]",2,23.373341,0.752714
2,"[38, 15, 87]",3,34.445109,0.758745
3,"[38, 15, 87, 76]",4,45.489959,0.761158
4,"[38, 15, 87, 76, 78]",5,56.708308,0.767189


In [ ]:
df_performance